# Statistical Analysis: Male vs Female Customer Sales

**Research Question:** Is there a statistically significant difference in average total sales between male and female customers?

---

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Load data
df = pd.read_csv('supermarket_sales.csv')
print(f"Dataset loaded: {df.shape[0]} transactions\n")

## 1. Descriptive Statistics

In [ ]:
# Separate data by gender
female_transactions = df[df['Gender'] == 'Female']['Total']
male_transactions = df[df['Gender'] == 'Male']['Total']

# Calculate statistics
female_mean = female_transactions.mean()
female_std = female_transactions.std()
female_n = len(female_transactions)

male_mean = male_transactions.mean()
male_std = male_transactions.std()
male_n = len(male_transactions)

female_total = female_transactions.sum()
male_total = male_transactions.sum()
total_sales = df['Total'].sum()

print("="*70)
print("DESCRIPTIVE STATISTICS")
print("="*70)
print(f"\n{'Gender':<10} {'Mean ($)':<15} {'Std Dev ($)':<15} {'Sample Size':<15}")
print("-"*70)
print(f"{'Female':<10} {female_mean:<15.2f} {female_std:<15.2f} {female_n:<15}")
print(f"{'Male':<10} {male_mean:<15.2f} {male_std:<15.2f} {male_n:<15}")

print(f"\n{'='*70}")
print("TOTAL SALES")
print("="*70)
print(f"Female: ${female_total:,.2f} ({female_total/total_sales*100:.2f}%)")
print(f"Male:   ${male_total:,.2f} ({male_total/total_sales*100:.2f}%)")
print(f"Difference: {abs(female_total/total_sales - male_total/total_sales)*100:.2f} percentage points")

## 2. Calculate Test Statistics

In [ ]:
# Step 1: Difference in sample means
mean_diff = female_mean - male_mean
print("="*70)
print("STEP 1: DIFFERENCE IN SAMPLE MEANS")
print("="*70)
print(f"x̄₁ - x̄₂ = {female_mean:.2f} - {male_mean:.2f} = {mean_diff:.2f}")

# Step 2: Pooled standard deviation
pooled_var = ((female_n - 1) * female_std**2 + (male_n - 1) * male_std**2) / (female_n + male_n - 2)
pooled_std = np.sqrt(pooled_var)
print(f"\n{'='*70}")
print("STEP 2: POOLED STANDARD DEVIATION")
print("="*70)
print(f"sp = √[((n₁-1)s₁² + (n₂-1)s₂²) / (n₁+n₂-2)]")
print(f"sp = √[(({female_n}-1)×{female_std:.2f}² + ({male_n}-1)×{male_std:.2f}²) / {female_n+male_n-2}]")
print(f"sp = {pooled_std:.2f}")

# Step 3: Standard error
se = pooled_std * np.sqrt(1/female_n + 1/male_n)
print(f"\n{'='*70}")
print("STEP 3: STANDARD ERROR")
print("="*70)
print(f"SE = sp × √(1/n₁ + 1/n₂)")
print(f"SE = {pooled_std:.2f} × √(1/{female_n} + 1/{male_n})")
print(f"SE = {pooled_std:.2f} × {np.sqrt(1/female_n + 1/male_n):.4f}")
print(f"SE = {se:.2f}")

# Step 4: 95% Confidence Interval
t_critical = 1.96  # For large samples, approximately normal
ci_lower = mean_diff - t_critical * se
ci_upper = mean_diff + t_critical * se
print(f"\n{'='*70}")
print("STEP 4: 95% CONFIDENCE INTERVAL")
print("="*70)
print(f"CI = (x̄₁ - x̄₂) ± t₀.₀₂₅,df × SE")
print(f"CI = {mean_diff:.2f} ± {t_critical} × {se:.2f}")
print(f"CI = {mean_diff:.2f} ± {t_critical * se:.2f}")
print(f"CI = [{ci_lower:.2f}, {ci_upper:.2f}]")
print(f"\nInterpretation: CI {'includes' if ci_lower <= 0 <= ci_upper else 'does not include'} 0")
if ci_lower <= 0 <= ci_upper:
    print("→ Suggests no significant difference")
else:
    print("→ Suggests significant difference exists")

## 3. Hypothesis Testing

In [ ]:
print("="*70)
print("HYPOTHESIS TEST")
print("="*70)
print("\nH₀: μ_Female = μ_Male (No difference in average transaction value)")
print("H₁: μ_Female ≠ μ_Male (Significant difference exists)")
print("\nSignificance level: α = 0.05")

# Calculate t-statistic
t_stat = mean_diff / se
df = female_n + male_n - 2

print(f"\n{'='*70}")
print("T-STATISTIC")
print("="*70)
print(f"t = (x̄₁ - x̄₂) / SE = {mean_diff:.2f} / {se:.2f} = {t_stat:.3f}")
print(f"df = n₁ + n₂ - 2 = {female_n} + {male_n} - 2 = {df}")

# Calculate p-value using scipy
t_test_result = stats.ttest_ind(female_transactions, male_transactions)
p_value = t_test_result.pvalue

print(f"\n{'='*70}")
print("P-VALUE")
print("="*70)
print(f"For two-tailed test with t = {t_stat:.3f} and df = {df}:")
print(f"p-value = {p_value:.4f}")

## 4. Effect Size

In [ ]:
# Cohen's d
cohens_d = mean_diff / pooled_std

print("="*70)
print("EFFECT SIZE (Cohen's d)")
print("="*70)
print(f"d = (x̄₁ - x̄₂) / sp = {mean_diff:.2f} / {pooled_std:.2f} = {cohens_d:.3f}")

if abs(cohens_d) < 0.2:
    effect = "Negligible"
elif abs(cohens_d) < 0.5:
    effect = "Small"
elif abs(cohens_d) < 0.8:
    effect = "Medium"
else:
    effect = "Large"

print(f"\nInterpretation: {effect} effect (d {'<' if abs(cohens_d) < 0.2 else '>='} 0.2)")

## 5. Decision and Conclusion

In [ ]:
print("="*70)
print("DECISION")
print("="*70)
print(f"\nDecision rule: Reject H₀ if p < 0.05")
print(f"\nSince p = {p_value:.4f} {'<' if p_value < 0.05 else '>'} 0.05:")

if p_value < 0.05:
    decision = "REJECT H₀"
    conclusion = f"There IS a statistically significant difference between female and male customer transaction values."
else:
    decision = "FAIL TO REJECT H₀"
    conclusion = f"There is NO statistically significant difference between female and male customer transaction values."

print(f"\n→ {decision}")

print(f"\n{'='*70}")
print("CONCLUSION")
print("="*70)
print(f"\n{conclusion}")
print(f"\nThe observed ${abs(mean_diff):.2f} difference {'is' if p_value < 0.05 else 'is not'} statistically significant.")

if p_value >= 0.05:
    print(f"This difference is likely attributable to random variation.")
else:
    print(f"Female customers spend ${abs(mean_diff):.2f} {'more' if mean_diff > 0 else 'less'} per transaction on average.")

print(f"\nEffect size: {effect} (Cohen's d = {cohens_d:.3f})")

## Summary Table

In [ ]:
print("="*70)
print("SUMMARY OF RESULTS")
print("="*70)
print(f"\n{'Statistic':<30} {'Value':<20}")
print("-"*70)
print(f"{'Difference in means':<30} ${mean_diff:.2f}")
print(f"{'Pooled std deviation':<30} ${pooled_std:.2f}")
print(f"{'Standard error':<30} ${se:.2f}")
print(f"{'T-statistic':<30} {t_stat:.3f}")
print(f"{'Degrees of freedom':<30} {df}")
print(f"{'P-value':<30} {p_value:.4f}")
print(f"{'Cohen\'s d':<30} {cohens_d:.3f} ({effect})")
print(f"{'95% CI':<30} [{ci_lower:.2f}, {ci_upper:.2f}]")
print(f"{'Decision':<30} {decision}")
print("="*70)